In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import time
import warnings
from wordcloud import WordCloud
%matplotlib inline
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [2]:
def get_text(url):
    p = urlopen(url)
    s = BeautifulSoup(p)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    a = len(soup.find_all('p'))
    text = ""
    for i in range(0, a):
        text = text + soup.find_all('p')[i].text + " "
        text = text.replace("\n"," ")
    return text

In [3]:
def get_links(url):
    page = urlopen(url)
    soup = BeautifulSoup(page)
    links = []
    for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
        links.append(link.get('href'))
    return links

In [4]:
def get_story_url(search_string):
    page = requests.get(search_string)
    soup = BeautifulSoup(page.content)
    links = soup.findAll("a")
    results = []
    for link in  soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):
        whole_thing = re.split(":(?=http)",link["href"].replace("/url?q=",""))
        results.append(whole_thing[0].split('&')[0])
    rv = []
    for result in results:
        if "www.vanityfair.com" in result:
            rv.append(result)
    return rv

In [5]:
def get_images(pics):
    images = []
    for pic in pics:
        if 'https' in pic:
            image = Image.open(urlopen(pic))
            images.append(image)
    return images

In [6]:
def get_url_set(years):
    search_string_root = "https://www.google.com/search?q=vanity+fair+cover+story"
    search_strings = []
    months = ['february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
    #years = [2015, 2016, 2017]
    for month in  months:
        for year in years:
            search_string = search_string_root + "+" + month + "+" + str(year)
            search_strings.append(search_string)

    # get story urls
    urls = []
    for search_string in search_strings:
        urls.extend(get_story_url(search_string))

    urls = list(set(urls))
    url_set = []
    for url in urls:
        if len(url) > 50:
            url_set.append(url)

    urls = url_set
    return urls

In [17]:
def get_details(url, data):
    p = urlopen(url)
    s = BeautifulSoup(p)
    img_tags = s.find_all('img')
    t = s.find_all('script', type="application/ld+json")
    d = eval(t[0].contents[0])
    
    author = d['creator'][0][0]
    try:
        photographer = d['creator'][0][1]
    except:
        pass
    try:
        stylist = d['creator'][0][2]
    except:
        pass
    date = d['dateCreated']
    cover_image = d['image']['url']
    tags = d['keywords']
    headline = d['headline']
    text = get_text(url)
    pics = get_images([img['src'] for img in img_tags])
    

    data['author'].append(author)
    try:
        data['photographer'].append(photographer)
    except:
        data['photographer'].append(" ")

    try:
        data['stylist'].append(stylist)
    except:
        data['stylist'].append("")
    data['date'].append(date)
    data['cover_image'].append(cover_image)
    data['tags'].append(tags)
    data['headline'].append(headline)
    data['text'].append(text)
    if pics:
        data['pics'].append(pics)
    else:
        data['pics'].append([])
    return data

In [8]:
%%time
## takes approx 7 minutes
data = {'author': [], 'photographer': [], 'stylist': [], 'date': [],
       'cover_image': [], 'tags': [], 'headline': [],
       'text': [], 'pics': []}

for url in get_url_set([2015,2016,2017]):
    if "rihanna" in url and "cuba-annie-leibovitz" in url:
        continue
    else:
        try:
            get_details(url,data)
        except:
            pass


CPU times: user 46.2 s, sys: 3.36 s, total: 49.5 s
Wall time: 7min 52s


In [9]:
%%time
## takes approx 11 minutes
data2 = {'author': [], 'photographer': [], 'stylist': [], 'date': [],
       'cover_image': [], 'tags': [], 'headline': [],
       'text': [], 'pics': []}

for url in get_url_set([2012,2013,2014]):
    if "rihanna" in url and "cuba-annie-leibovitz" in url:
        continue
    else:
        try:
            get_details(url,data2)
        except:
            pass


CPU times: user 48.4 s, sys: 3.58 s, total: 51.9 s
Wall time: 7min 44s


In [10]:
df1 = pd.DataFrame(data)
df2 = pd.DataFrame(data2)
df = df1.append(df2)

In [11]:
df = df[df['author'] != "Vanity Fair"]
df = df.replace('', np.nan, regex=True)


df = df.dropna(subset=['text'])
df['text'] = df['text'].str[:-455] # remove Conde Nast user agreement from each interview

In [14]:
male_authors = ["Todd Purdum", "Sam Kashner", "Rich Cohen", "Mitch Glazer", 
                "Michael Schulman", "Josh Duboff", "Jim Windolf", "James Fox", 
                "Jack Deligter, Jaime Lalinde and Mickey Stanley", "Alec Baldwin", 
                "Alex Beggs", "Bruce Handy", "Buzz Bissinger", "David Kamp", 
                "Derek Blasberg", "Fred Burton"] 

def author_gender(row):
    global male_authors
    if row['author'] in male_authors:
        return 'male'
    return 'female'

df['author_gender'] = df.apply (lambda row: author_gender(row), axis=1 )

def cover(row):
    if 'cover story' in row['tags']:
        return "cover"
    return "DELETE"
df['cover'] = df.apply (lambda row: cover(row), axis=1 )
df = df[df['cover'] == "cover"]



df['celeb_gender'] = [0,0,0,999,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,999,0,1,0,0,0,0,0]
df = df[df['celeb_gender']< 2]

mask = df.celeb_gender ==  0
column_name = 'celeb_gender'
df.loc[mask, column_name] = 'female'
df.groupby('celeb_gender').count()[['author']]

mask = df.celeb_gender ==  1
column_name = 'celeb_gender'
df.loc[mask, column_name] = 'male'
df.groupby('celeb_gender').count()[['author']]

ValueError: Length of values does not match length of index

In [15]:
len(df)

35

In [ ]:
df.groupby('author_gender').count()[['author']]

In [ ]:
pd.crosstab(df.author_gender, df.celeb_gender)

In [ ]:
df['year'] = df['date'].astype(str).str[0:4]
# check if GQ years add up
df.groupby('year').count()[['author']]

In [ ]:
df['text_length'] = df['text'].str.len()

In [ ]:
df.dropna(subset=['text'], inplace = True)
df_fm = df[(df['author_gender'] == "female") & (df['celeb_gender'] == "male")]
df_fm['pair'] = "Female-Male"
df_ff = df[(df['author_gender'] == "female") & (df['celeb_gender'] == "female")]
df_ff['pair'] = "Female-Female"
df_mf = df[(df['author_gender'] == "male") & (df['celeb_gender'] == "female")]
df_mf['pair'] = "Male-Female"
df_mm = df[(df['author_gender'] == "male") & (df['celeb_gender'] == "male")]
df_mm['pair'] = "Male-Male"

df_f = pd.concat([df_fm, df_ff])
df_m = pd.concat([df_mf, df_mm])

plot_data = pd.concat([df_fm,df_ff, df_mf, df_mm])


In [ ]:
sns.distplot(df['text_length'], hist = False, rug = True, kde = True)

In [ ]:
sns.factorplot(kind ='box',        # Boxplot
               y = 'text_length',       # Y-axis - values for boxplot
               x ='author_gender',        # X-axis - first factor
               data = df,
               size = 8,            # Figure size (x100px)      
               aspect = 1,        # Width = size * aspect 
               legend_out = False)  # Make legend inside the plot

In [ ]:
sns.factorplot(x = "pair", 
               y="text_length", 
               data=plot_data, 
               kind="box",
               size = 8,
               aspect = 1
              )


In [ ]:
%%time
original = df
original['pair']

In [ ]:
%%time
df['tokens'] = df['text'].apply(lambda x: [nltk.word_tokenize(s) for s in nltk.sent_tokenize(x)])
from nltk.corpus import stopwords
# tokenize
flatten = lambda l: [item for sublist in l for item in sublist]
tokens = flatten(flatten(df['tokens'].tolist()))
# remove stop words
stopwords = stopwords.words('english')
clean_tokens = []
for token in tokens:
    t = token.lower()
    if t not in stopwords and len(t) > 1: # remove stopwords and punctuation
        clean_tokens.append(t)
freq = nltk.FreqDist(clean_tokens)
tokens_all = clean_tokens
freq.plot(20,cumulative=False)

In [ ]:
%%time
df = df_mm
df['tokens'] = df['text'].apply(lambda x: [nltk.word_tokenize(s) for s in nltk.sent_tokenize(x)])
from nltk.corpus import stopwords
# tokenize
flatten = lambda l: [item for sublist in l for item in sublist]
tokens = flatten(flatten(df['tokens'].tolist()))
# remove stop words
stopwords = stopwords.words('english')
clean_tokens = []
for token in tokens:
    t = token.lower()
    if t not in stopwords and len(t) > 1: # remove stopwords and punctuation
        clean_tokens.append(t)
freq = nltk.FreqDist(clean_tokens)
tokens_mm = clean_tokens


freq.plot(20,cumulative=False)

In [ ]:
%%time
df = df_mf
df['tokens'] = df['text'].apply(lambda x: [nltk.word_tokenize(s) for s in nltk.sent_tokenize(x)])
from nltk.corpus import stopwords
# tokenize
flatten = lambda l: [item for sublist in l for item in sublist]
tokens = flatten(flatten(df['tokens'].tolist()))
tokens_f = tokens
# remove stop words
stopwords = stopwords.words('english')
clean_tokens = []
for token in tokens:
    t = token.lower()
    if t not in stopwords and len(t) > 1: # remove stopwords and punctuation
        clean_tokens.append(t)
freq = nltk.FreqDist(clean_tokens)
tokens_mf = clean_tokens


freq.plot(20,cumulative=False)

In [ ]:
%%time
df = df_ff
df['tokens'] = df['text'].apply(lambda x: [nltk.word_tokenize(s) for s in nltk.sent_tokenize(x)])
from nltk.corpus import stopwords
# tokenize
flatten = lambda l: [item for sublist in l for item in sublist]
tokens = flatten(flatten(df['tokens'].tolist()))
tokens_f = tokens
# remove stop words
stopwords = stopwords.words('english')
clean_tokens = []
for token in tokens:
    t = token.lower()
    if t not in stopwords and len(t) > 1: # remove stopwords and punctuation
        clean_tokens.append(t)
freq = nltk.FreqDist(clean_tokens)
tokens_ff = clean_tokens


freq.plot(20,cumulative=False)

In [ ]:
%%time
df = df_mm
df['tokens'] = df['text'].apply(lambda x: [nltk.word_tokenize(s) for s in nltk.sent_tokenize(x)])
from nltk.corpus import stopwords
# tokenize
flatten = lambda l: [item for sublist in l for item in sublist]
tokens = flatten(flatten(df['tokens'].tolist()))
tokens_f = tokens
# remove stop words
stopwords = stopwords.words('english')
clean_tokens = []
for token in tokens:
    t = token.lower()
    if t not in stopwords and len(t) > 1: # remove stopwords and punctuation
        clean_tokens.append(t)
freq = nltk.FreqDist(clean_tokens)
tokens_fm = clean_tokens


freq.plot(20,cumulative=False)

In [ ]:
%%time
# see collocations
text = nltk.Text(tokens_all)
text_mm = nltk.Text(tokens_mm)
text_ff = nltk.Text(tokens_ff)
text_mf = nltk.Text(tokens_mf)
text_fm = nltk.Text(tokens_fm)

In [ ]:
text.collocations()

In [ ]:
text_mm.collocations()

In [ ]:
text_ff.collocations()

In [ ]:
text_mf.collocations()

In [ ]:
text_fm.collocations()

In [ ]:
texts = [text, text_mm, text_ff, text_mf, text_fm]
words = ['sexy', 'hot', 'pretty','beautiful','woman']
for t in texts:
    for word in words:
        if t == text:
            print("All" + " " + word)
        if t == text_m:
            print("Male" + " " + word)
        if t == text_f:
            print("Female" + " " + word)
        if t.common_contexts([word]):
                print(t.common_contexts([word]))
        

In [18]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(tokens_all, 10)
finder.apply_freq_filter(10)
finder.score_ngrams (bigram_measures.likelihood_ratio)[:10]

NameError: name 'tokens_all' is not defined

In [ ]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
whTrigrams = nltk.collocations.TrigramCollocationFinder.from_words(tokens_all)
whTrigrams.score_ngrams(trigram_measures.student_t)[:20]

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(tokens_mm, 10)
finder.apply_freq_filter(10)
finder.score_ngrams (bigram_measures.likelihood_ratio)[:10]

In [ ]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
whTrigrams = nltk.collocations.TrigramCollocationFinder.from_words(tokens_mm)
whTrigrams.score_ngrams(trigram_measures.student_t)[:20]

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(tokens_ff, 10)
finder.apply_freq_filter(10)
finder.score_ngrams (bigram_measures.likelihood_ratio)[:10]

In [ ]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
whTrigrams = nltk.collocations.TrigramCollocationFinder.from_words(tokens_ff)
whTrigrams.score_ngrams(trigram_measures.student_t)[:20]

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(tokens_mf, 10)
finder.apply_freq_filter(10)
finder.score_ngrams (bigram_measures.likelihood_ratio)[:10]

In [ ]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
whTrigrams = nltk.collocations.TrigramCollocationFinder.from_words(tokens_mf)
whTrigrams.score_ngrams(trigram_measures.student_t)[:20]

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(tokens_fm, 10)
finder.apply_freq_filter(10)
finder.score_ngrams (bigram_measures.likelihood_ratio)[:10]

In [ ]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
whTrigrams = nltk.collocations.TrigramCollocationFinder.from_words(tokens_fm)
whTrigrams.score_ngrams(trigram_measures.student_t)[:20]

In [ ]:
corpora = [tokens_mm, tokens_mf, tokens_ff, tokens_fm]
stopwords = stopwords.words('english')
corpora_s = []
corpora_nons = []
for corpus in corpora:
    s = []
    nons = []
    for word in corpus:
        if word in stopwords:
            s.append(word)
        else:
            nons.append(word)
    corpora_s.append(s)
    corpora_nons.append(nons)
    
def kl_divergence(X, Y):
    P = X.copy()
    Q = Y.copy()
    P.columns = ['P']
    Q.columns = ['Q']
    df = Q.join(P).fillna(0)
    p = df.iloc[:,1]
    q = df.iloc[:,0]
    D_kl = scipy.stats.entropy(p, q)
    return D_kl

def chi2_divergence(X,Y):
    P = X.copy()
    Q = Y.copy()
    P.columns = ['P']
    Q.columns = ['Q']
    df = Q.join(P).fillna(0)
    p = df.iloc[:,1]
    q = df.iloc[:,0]
    return scipy.stats.chisquare(p, q).statistic

def Divergence(corpus1, corpus2, difference="KL"):
    """Difference parameter can equal KL, Chi2, or Wass"""
    freqP = nltk.FreqDist(corpus1)
    P = pandas.DataFrame(list(freqP.values()), columns = ['frequency'], index = list(freqP.keys()))
    freqQ = nltk.FreqDist(corpus2)
    Q = pandas.DataFrame(list(freqQ.values()), columns = ['frequency'], index = list(freqQ.keys()))
    if difference == "KL":
        return kl_divergence(P, Q)
    elif difference == "Chi2":
        return chi2_divergence(P, Q)
    elif difference == "KS":
        try:
            return scipy.stats.ks_2samp(P['frequency'], Q['frequency']).statistic
        except:
            return scipy.stats.ks_2samp(P['frequency'], Q['frequency'])
    elif difference == "Wasserstein":
        try:
            return scipy.stats.wasserstein_distance(P['frequency'], Q['frequency'], u_weights=None, v_weights=None).statistic
        except:
            return scipy.stats.wasserstein_distance(P['frequency'], Q['frequency'], u_weights=None, v_weights=None)

In [ ]:
import lucem_illud 
import requests #for http requests
import nltk #the Natural Language Toolkit
import pandas #gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import wordcloud #Makes word clouds
import numpy as np #For divergences/distances
import scipy #For divergences/distances
import seaborn as sns #makes our plots look nicer
import sklearn.manifold #For a manifold plot
from nltk.corpus import stopwords #For stopwords
import json #For API responses
import urllib.parse #For joining urls
import gzip
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
L = []
for p in corpora:
    l = []
    for q in corpora:
        l.append(Divergence(p,q, difference = 'KL'))
    L.append(l)
M = np.array(L)
fig = plt.figure()
div = pandas.DataFrame(M, columns = ['MM','MF','FF','FM'], index = ['MM','MF','FF','FM'])
ax = sns.heatmap(div)
plt.show()

In [ ]:
fileids = ['MM','MF','FF','FM']
L = []
for p in corpora_nons:
    l = []
    for q in corpora_nons:
        l.append(Divergence(p,q, difference='KL'))
    L.append(l)
M = np.array(L)
fig = plt.figure()
div = pandas.DataFrame(M, columns = fileids, index = fileids)
ax = sns.heatmap(div)
plt.show()

In [ ]:
L = []
for p in corpora_nons:
    l = []
    for q in corpora_nons:
        l.append(Divergence(p,q, difference='KS'))
    L.append(l)
M = np.array(L)
fig = plt.figure()
div = pandas.DataFrame(M, columns = fileids, index = fileids)
ax = sns.heatmap(div)
plt.show()

In [ ]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
#Make sure you update it before starting this notebook
import lucem_illud #pip install -U git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git


#All these packages need to be installed from pip
#These are all for the cluster detection
import sklearn
import sklearn.feature_extraction.text
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.datasets
import sklearn.cluster
import sklearn.decomposition
import sklearn.metrics

import scipy #For hierarchical clustering and some visuals
#import scipy.cluster.hierarchy
import gensim#For topic modeling
import nltk #the Natural Language Toolkit
import requests #For downloading our datasets
import numpy as np #for arrays
import pandas #gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import matplotlib.cm #Still for graphics
import seaborn as sns #Makes the graphics look nicer

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook.
#Also you can ignore the warning, it
%matplotlib inline

import itertools
import json
import gzip
import pandas as pd
import datetime
import warnings


In [ ]:
%%time
df = plot_data
# create count vector
ngCountVectorizer = sklearn.feature_extraction.text.CountVectorizer()
newsgroupsVects = ngCountVectorizer.fit_transform(df['text'])

# tf-idf
newsgroupsTFTransformer = sklearn.feature_extraction.text.TfidfTransformer().fit(newsgroupsVects)
newsgroupsTF = newsgroupsTFTransformer.transform(newsgroupsVects)

# # vectorize
ngTFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(stop_words='english', norm='l2')
newsgroupsTFVects = ngTFVectorizer.fit_transform(df['text'])


# # k-means
numClusters = 4

km = sklearn.cluster.KMeans(n_clusters=numClusters, init='k-means++')
km.fit(newsgroupsTFVects)

In [ ]:
terms = ngTFVectorizer.get_feature_names()
print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(numClusters):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])
    print('\n')

In [ ]:
PCA = sklearn.decomposition.PCA
pca = PCA(n_components = 2).fit(newsgroupsTFVects.toarray())
reduced_data = pca.transform(newsgroupsTFVects.toarray())

colordict = {'Male-Male': 'red','Male-Female': 'orange','Female-Female': 'green','Female-Male': 'blue'}
colors = [colordict[c] for c in df['pair']]
components = pca.components_
keyword_ids = list(set(order_centroids[:,:10].flatten())) #Get the ids of the most distinguishing words(features) from your kmeans model.
words = [terms[i] for i in keyword_ids]#Turn the ids into words.
x = components[:,keyword_ids][0,:] #Find the coordinates of those words in your biplot.
y = components[:,keyword_ids][1,:]

In [ ]:
fig = plt.figure(figsize = (10,6))
ax = fig.add_subplot(111)
ax.set_frame_on(False)
ax.scatter(reduced_data[:, 0], reduced_data[:, 1], color = colors, alpha = 0.3, label = colors)
newsgroupsCategories = ['Male-Male', 'Male-Female', 'Female-Female','Female-Male']
colors_p = [colordict[newsgroupsCategories[l]] for l in km.labels_]
for i, word in enumerate(words):
    ax.annotate(word, (x[i],y[i]))
plt.xticks(())
plt.yticks(())
plt.title('True Classes')
plt.show()

In [ ]:
fig = plt.figure(figsize = (10,6))
ax = fig.add_subplot(111)
ax.set_frame_on(False)
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], color = colors_p, alpha = 0.5)
plt.xticks(())
plt.yticks(())
plt.title('Predicted Clusters\n k = 4')
plt.show()

In [ ]:
def plotSilhouette(n_clusters, X):
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize = (15,5))
    
    ax1.set_xlim([-0.1, 1])
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])
    clusterer = sklearn.cluster.KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)
    
    silhouette_avg = sklearn.metrics.silhouette_score(X, cluster_labels)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = sklearn.metrics.silhouette_samples(X, cluster_labels)

    y_lower = 10
    
    for i in range(n_clusters):
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = matplotlib.cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        y_lower = y_upper + 10
    
    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = matplotlib.cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(reduced_data[:, 0], reduced_data[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors)

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    projected_centers = pca.transform(centers)
    # Draw white circles at cluster centers
    ax2.scatter(projected_centers[:, 0], projected_centers[:, 1],
                marker='o', c="white", alpha=1, s=200)

    for i, c in enumerate(projected_centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1, s=50)

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("PC 1")
    ax2.set_ylabel("PC 2")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')
    plt.show()
    print("For n_clusters = {}, The average silhouette_score is : {:.3f}".format(n_clusters, silhouette_avg))
    return silhouette_avg

In [ ]:
X = newsgroupsTFVects.toarray()


In [ ]:
%%time
df = plot_data
# create count vector
ngCountVectorizer = sklearn.feature_extraction.text.CountVectorizer()
newsgroupsVects = ngCountVectorizer.fit_transform(df['text'])

# tf-idf
newsgroupsTFTransformer = sklearn.feature_extraction.text.TfidfTransformer().fit(newsgroupsVects)
newsgroupsTF = newsgroupsTFTransformer.transform(newsgroupsVects)

# # vectorize
ngTFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(stop_words='english', norm='l2')
newsgroupsTFVects = ngTFVectorizer.fit_transform(df['text'])


# # k-means
numClusters = 2

km = sklearn.cluster.KMeans(n_clusters=numClusters, init='k-means++')
km.fit(newsgroupsTFVects)

In [ ]:
PCA = sklearn.decomposition.PCA
pca = PCA(n_components = 2).fit(newsgroupsTFVects.toarray())
reduced_data = pca.transform(newsgroupsTFVects.toarray())

colordict = {'male': 'red','female': 'blue'}
colors = [colordict[c] for c in df['author_gender']]
components = pca.components_
keyword_ids = list(set(order_centroids[:,:10].flatten())) #Get the ids of the most distinguishing words(features) from your kmeans model.
words = [terms[i] for i in keyword_ids]#Turn the ids into words.
x = components[:,keyword_ids][0,:] #Find the coordinates of those words in your biplot.
y = components[:,keyword_ids][1,:]


In [ ]:
fig = plt.figure(figsize = (10,6))
ax = fig.add_subplot(111)
ax.set_frame_on(False)
ax.scatter(reduced_data[:, 0], reduced_data[:, 1], color = colors, alpha = 0.3, label = colors)
newsgroupsCategories = ['male', 'female']
colors_p = [colordict[newsgroupsCategories[l]] for l in km.labels_]
for i, word in enumerate(words):
    ax.annotate(word, (x[i],y[i]))
plt.xticks(())
plt.yticks(())
plt.title('True Classes')
plt.show()

In [ ]:
fig = plt.figure(figsize = (10,6))
ax = fig.add_subplot(111)
ax.set_frame_on(False)
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], color = colors_p, alpha = 0.5)
plt.xticks(())
plt.yticks(())
plt.title('Predicted Clusters\n k = 4')
plt.show()

In [ ]:
X = newsgroupsTFVects.toarray()
plotSilhouette(2, X)

plotSilhouette(3, X)

plotSilhouette(4, X)


plotSilhouette(5, X)


In [ ]:
terms = ngTFVectorizer.get_feature_names()
print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(numClusters):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])
    print('\n')

In [ ]:
newsgroupsTFVects[:100].todense()


In [ ]:
newsgroupsCoocMat = newsgroupsTFVects * newsgroupsTFVects.T
#set the diagonal to 0 since we don't care how similar texts are to themselves
newsgroupsCoocMat.setdiag(0)

In [ ]:

df = df.reset_index(drop=True)

selectIndices = []
indexToCat = []
for c in set(df['pair']):
    selectIndices += list(df[df['pair'] == c].index)[:1000]
    indexToCat += [c] * 50
    #.groupby('category').sum()

In [ ]:
subCoocMat = newsgroupsCoocMat[selectIndices,:][:,selectIndices]
linkage_matrix = scipy.cluster.hierarchy.ward(subCoocMat.toarray())

In [ ]:
dendDat = scipy.cluster.hierarchy.dendrogram(linkage_matrix, get_leaves=True)


In [ ]:
# label and see

In [ ]:
len(plot_data)

In [ ]:
#Apply our functions
senReleasesTraining = plot_data

senTFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_df=100, min_df=2, stop_words='english', norm='l2')
senTFVects = senTFVectorizer.fit_transform(senReleasesTraining['text'])
#senTFVectorizer.vocabulary_.get('senat', 'Missing "Senate"')

senReleasesTraining['tokenized_text'] = senReleasesTraining['text'].apply(lambda x: nltk.word_tokenize(x))
senReleasesTraining['normalized_tokens'] = senReleasesTraining['tokenized_text'].apply(lambda x: lucem_illud.normalizeTokens(x, 
                                                                                                                            stopwordLst = lucem_illud.stop_words_basic, 
                                                                                                                            stemmer = lucem_illud.stemmer_basic))
def dropMissing(wordLst, vocab):
    return [w for w in wordLst if w in vocab]

senReleasesTraining['reduced_tokens'] = senReleasesTraining['normalized_tokens'].apply(lambda x: dropMissing(x, senTFVectorizer.vocabulary_.keys()))

dictionary = gensim.corpora.Dictionary(senReleasesTraining['reduced_tokens'])
corpus = [dictionary.doc2bow(text) for text in senReleasesTraining['reduced_tokens']]

gensim.corpora.MmCorpus.serialize('senate.mm', corpus)
senmm = gensim.corpora.MmCorpus('senate.mm')

senlda = gensim.models.ldamodel.LdaModel(corpus=senmm, id2word=dictionary, num_topics=10, alpha='auto', eta='auto')
#senReleasesTraining.reset_index(inplace = True)
sen1Bow = dictionary.doc2bow(senReleasesTraining['reduced_tokens'][0])
sen1lda = senlda[sen1Bow]
print("The topics of the text: {}".format(senReleasesTraining['headline'][0]))
print("are: {}".format(sen1lda))

In [ ]:
#senReleasesTraining.reset_index(inplace = True)
sen1Bow = dictionary.doc2bow(senReleasesTraining['reduced_tokens'][0])
sen1lda = senlda[sen1Bow]
print("The topics of the text: {}".format(senReleasesTraining['headline'][0]))
print("are: {}".format(sen1lda))

In [ ]:
ldaDF = pandas.DataFrame({
        'name' : senReleasesTraining['headline'],
        'topics' : [senlda[dictionary.doc2bow(l)] for l in senReleasesTraining['reduced_tokens']]
    })

In [ ]:
#Dict to temporally hold the probabilities
topicsProbDict = {i : [0] * len(ldaDF) for i in range(senlda.num_topics)}

#Load them into the dict
for index, topicTuples in enumerate(ldaDF['topics']):
    for topicNum, prob in topicTuples:
        topicsProbDict[topicNum][index] = prob

#Update the DataFrame
for topicNum in range(senlda.num_topics):
    ldaDF['topic_{}'.format(topicNum)] = topicsProbDict[topicNum]

ldaDF[1::100]

In [ ]:
ldaDFV = ldaDF[:10][['topic_%d' %x for x in range(3)]]
ldaDFVisN = ldaDF[:10][['name']]
ldaDFVis = ldaDFV.as_matrix(columns=None)
ldaDFVisNames = ldaDFVisN.as_matrix(columns=None)
ldaDFV

In [ ]:
senlda.show_topic(1)


In [ ]:
senlda.show_topic(0)


In [ ]:
senlda.show_topic(2)


In [ ]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
#Make sure you update it before starting this notebook
import lucem_illud #pip install -U git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git

#All these packages need to be installed from pip
#For NLP
import nltk

import numpy as np #For arrays
import pandas #Gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import seaborn #Makes the graphics look nicer

#Displays the graphs
import graphviz #You also need to install the command line graphviz

#These are from the standard library
import os.path
import zipfile
import subprocess
import io
import tempfile
#import lucem_illud.stanford as stanford
#lucem_illud.setupStanfordNLP()

%matplotlib inline

In [ ]:
print("sdf")